In [13]:
import pandas as pd
from fullcontact import FullContactClient
import numpy as np

df = pd.read_csv(r'C:\Users\aacjp\Spencer\datasets\MockHyros.csv')

In [14]:
df

,lead,closed,monthly traffic spend,domain,business model,monthly revenue,% paid traffic,average order value,monthly customers,visits,...,monthly hardcosts,ad revenue,roas,cac,profit,n links lp,n words lp,trigger words lp,lpc,profit margin
0,FrankKern.com,1,54437,entrepreneurship,info,583310,12.30,3000,194,86590,...,18666.0,7.174713e+04,1.317985,280.0,510207.0,2.0,2140.0,55.0,2198.00000,87.47
1,ImpactTheory.Com,1,126095,self-development,media,250000,12.23,97,25773,147290,...,8000.0,3.057500e+04,0.242476,5.0,115905.0,NaN,NaN,NaN,10787.69697,46.36
2,digitalmillionairesecrets.com,1,139711,entrepreneurship,info,1666667,82.83,997,1672,49999,...,53333.0,1.380500e+06,9.881112,84.0,1473623.0,3.0,108.0,1.0,113.00000,88.42
3,mastermind.com,1,220950,entrepreneurship,software,2877110,3.91,297,9687,807270,...,1726266.0,1.124950e+05,0.509142,23.0,929894.0,NaN,NaN,NaN,10787.69697,32.32
4,ShopaNova.com,1,100000,ecommerce,software,333333,38.51,2000,167,16700,...,200000.0,1.283670e+05,1.283670,599.0,33333.0,1.0,14.0,1.0,17.00000,10.00
5,SamCart.io,1,218815,ecommerce,software,750000,34.34,99,7576,1950000,...,450000.0,2.575500e+05,1.177022,29.0,81185.0,1.0,71.0,2.0,75.00000,10.82
6,Clickfunnels.com,1,3540166,marketing,software,11391660,59.23,297,38356,13070000,...,6834996.0,6.747280e+06,1.905922,92.0,1016498.0,14.0,3094.0,60.0,3169.00000,8.92
7,billygeneismarketing.com,1,116155,entrepreneurship,info,399984,37.92,197,2030,203000,...,12799.0,1.516740e+05,1.305790,57.0,271030.0,1.0,42.0,3.0,47.00000,67.76
8,kinobody.com,1,348603,fitness,info,1045809,15.99,69,15157,365360,...,33200.0,1.672250e+05,0.479700,11.0,664006.0,1.0,32.0,1.0,35.00000,63.49
9,Ladyboss.com,1,521329,fitness,info,949962,37.70,1000,950,482210,...,30399.0,4.018340e+05,0.686967,549.0,561427.0,5.0,287.0,5.0,298.00000,59.01


The Hack: http://sangaline.com/post/advanced-web-scraping-tutorial/

https://platform.fullcontact.com/developers/api-keys